<a href="https://colab.research.google.com/github/jmoriwa/ai_agents_workspace/blob/main/smarter_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#AI agent (with memory)

In [4]:
pip install -U langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.0/55.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 13.5 MB/s eta 0:00:00


In [5]:
pip install -U langchain-openai


In [6]:
import os
from typing import TypedDict, List, Union
from langchain_core.messages import HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END


In [16]:
from google.colab import userdata
api_key = userdata.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = api_key

class AgentState(TypedDict):
  messages: List[Union[HumanMessage, AIMessage]]

llm = ChatOpenAI(model="gpt-4o")


In [20]:
def process(state: AgentState) -> AgentState:
  """ This node will solve the request you input"""

  response = llm.invoke(state["messages"])

  state["messages"].append(AIMessage(content=response.content))
  print(f"\nAI: {response.content}")
  print("CURRENT STATE: ", state["messages"])

  return state

In [21]:
graph = StateGraph(AgentState)
graph.add_node("process", process)
graph.add_edge(START, "process")
graph.add_edge("process", END)
agent = graph.compile()

conversation_history = []


In [23]:
user_input = input("Enter: ")

while user_input != "exit":
  conversation_history.append(HumanMessage(content=user_input))
  result = agent.invoke({"messages": conversation_history})


  conversation_history = result["messages"]

  user_input = input("Enter: ")


Enter: What is my name

AI: Your name is Valentino.
CURRENT STATE:  [HumanMessage(content='Hi, nice to meet you', additional_kwargs={}, response_metadata={}), AIMessage(content='Hi! Nice to meet you too. How can I assist you today?', additional_kwargs={}, response_metadata={}), HumanMessage(content='My name is valentino', additional_kwargs={}, response_metadata={}), AIMessage(content='Nice to meet you, Valentino! How can I help you today?', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is my name', additional_kwargs={}, response_metadata={}), AIMessage(content='Your name is Valentino.', additional_kwargs={}, response_metadata={})]
Enter: exit
